In [17]:

# Step 1: Install and Import Libraries
!pip install transformers sentencepiece --quiet
print("Libraries installed.")

from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import textwrap

print("Libraries imported successfully.")

# Step 1: Load T5 Model for Summarization

t5_model_name = "t5-base"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)

print(f"✅ T5 Model ('{t5_model_name}') is loaded and ready!")

# Summarizer Function
def generate_summary(text, min_len=30, max_len=80, beams=5):
    input_text = "summarize: " + text.strip().replace("\n", " ")
    inputs = t5_tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

    summary_ids = t5_model.generate(
        inputs,
        max_length=max_len,
        min_length=min_len,
        num_beams=beams,
        no_repeat_ngram_size=3,
        length_penalty=2.0,
        early_stopping=True
    )
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example 1: Summarization of Electrical Text

sample_text = """
A transformer is an electrical device that transfers electrical energy between two or more circuits
through electromagnetic induction. Transformers are used to increase or decrease the alternating
voltages in electric power applications. They are essential in the transmission and distribution of
electrical energy, allowing efficient transfer of power over long distances while minimizing energy
losses.
"""

summary = generate_summary(sample_text, min_len=30, max_len=80, beams=5)

print("----------- Sample text -----------")
print(textwrap.fill(sample_text, width=100))
print("\n✨---------- T5 SUMMARY -----------✨")
print(textwrap.fill(summary, width=100))

# Example 2: Interactive Input + Analysis

input_text = "The synchronous motor operates at constant speed irrespective of the load applied to it."
generated_summary = generate_summary(input_text, min_len=25, max_len=70, beams=5)

# Analysis
original_word_count = len(input_text.split())
summary_word_count = len(generated_summary.split())
reduction = 100 - (summary_word_count / original_word_count * 100)

print("----------- YOUR INPUT TEXT -----------")
print(textwrap.fill(input_text, width=100))
print("\n✨---------- GENERATED SUMMARY -----------✨")
print(textwrap.fill(generated_summary, width=100))
print("\n📊---------- ANALYSIS -----------📊")
print(f"Original Word Count: {original_word_count}")
print(f"Summary Word Count: {summary_word_count}")
print(f"Text Reduction: {reduction:.1f}%")

# Step 2: Load PEGASUS Model for Paraphrasing

paraphrase_model_name = "tuner007/pegasus_paraphrase"
pegasus_tokenizer = PegasusTokenizer.from_pretrained(paraphrase_model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(paraphrase_model_name)

print(f"✅ PEGASUS Model ('{paraphrase_model_name}') is loaded and ready!")

# Paraphraser Function
def generate_paraphrases(text, num_return=5, beams=9):
    inputs = pegasus_tokenizer.encode(text, return_tensors="pt", truncation=True)
    paraphrase_ids = pegasus_model.generate(
        inputs,
        max_length=60,
        num_beams=beams,
        num_return_sequences=num_return,
        early_stopping=True
    )
    return pegasus_tokenizer.batch_decode(paraphrase_ids, skip_special_tokens=True)

# Example 3: Paraphrasing

original_sentence = "Learning new skills is essential for career growth."
generated_paraphrases = generate_paraphrases(original_sentence, num_return=5, beams=9)

print("----------- ORIGINAL SENTENCE -----------")
print(f"'{original_sentence}'\n")
print(f"🤖---------- {len(generated_paraphrases)} GENERATED PARAPHRASES (Quality: 4) ----------🤖")
for i, p in enumerate(generated_paraphrases):
    print(f"  {i+1}. {p}")


✅ Libraries installed.
✅ Libraries imported successfully.
✅ T5 Model ('t5-base') is loaded and ready!
----------- Sample text -----------
 A transformer is an electrical device that transfers electrical energy between two or more circuits
through electromagnetic induction. Transformers are used to increase or decrease the alternating
voltages in electric power applications. They are essential in the transmission and distribution of
electrical energy, allowing efficient transfer of power over long distances while minimizing energy
losses.

✨---------- T5 SUMMARY -----------✨
transformers are essential in the transmission and distribution of electrical energy . they allow
efficient transfer of power over long distances while minimizing energy losses .
----------- YOUR INPUT TEXT -----------
The synchronous motor operates at constant speed irrespective of the load applied to it.

✨---------- GENERATED SUMMARY -----------✨
the synchronous motor operates at constant speed irrespective of th

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ PEGASUS Model ('tuner007/pegasus_paraphrase') is loaded and ready!
----------- ORIGINAL SENTENCE -----------
'Learning new skills is essential for career growth.'

🤖---------- 5 GENERATED PARAPHRASES (Quality: 9) ----------🤖
  1. It's important to learn new skills for career growth.
  2. It's important for career growth to learn new skills.
  3. Career growth can be achieved by learning new skills.
  4. Career growth is dependent on learning new skills.
  5. New skills are needed for career growth.
